In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timezone
import io
import json
import pytz
from IPython.core.display import HTML

# Then import the key modules from the LUSID package (i.e. The LUSID SDK)
import lusid as lu
import lusid.api as la
import lusid.models as lm

# And use absolute imports to import key functions from Lusid-Python-Tools and other helper package

from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.jupyter_tools import StopExecution
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
)

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_status = pd.DataFrame(
    api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict()
)

display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.7607.0,0.5.2343,"{'relation': 'RequestLogs', 'href': 'http://om..."


# Read in and format data

In [2]:
scope = "srs"
df = pd.read_csv("Data.csv")
df["Date"] = pd.to_datetime(df["Date"], utc=True)
df =df.round(0)

## Define data ID

In [3]:
srs_api = api_factory.build(lu.StructuredResultDataApi)
code= "CashProjections" 
srs_ids = [
    lm.StructuredResultDataId(
        source="Client", 
        code=code, 
        effective_at=datetime(2021, 1, 27, tzinfo=timezone.utc),
        result_type="Raw"
    )
]

In [13]:
version= "1.01"
srs_ids=[]
for i in set(df['Port']):
    code= "CashProjections_" + i
    
    srs_id = lm.StructuredResultDataId(
        source="Client", 
        code=code, 
        effective_at=datetime(2021, 1, 27, tzinfo=timezone.utc),
        result_type="Raw")
    
    srs_ids.append(srs_id)
    
    df1 = df[df['Port'] == i]
    
    file_path= './data/' + code
    
    df1.to_csv(file_path)
    
    with open(file_path, 'r') as myfile: 
        data = myfile.read()
        
    request_body={
    i: lm.UpsertStructuredResultDataRequest(
    id=srs_id,
    data=lm.StructuredResultData(
    document_format="csv",
    version=version,
    name="Data file",
    document=data
        )
      )
    }

    result = srs_api.upsert_structured_result_data(
    scope=scope,
    request_body=request_body)

## Return Data from store

In [5]:
navs = srs_api.get_structured_result_data(
        scope=scope, 
        request_body={
            "key1": srs_ids
        }
    )
    
s = io.StringIO(navs.values["key1"].document)
    
nav_doc = pd.read_csv(s)

In [28]:
ports= df['Port'].unique()
for i, x in enumerate(ports):
    code= "CashProjections_" + x
    result = srs_api.get_structured_result_data(
    scope=scope, 
    request_body={
        x: srs_ids[i]
        }
        )
    s = io.StringIO(result.values[x].document)
    doc = pd.read_csv(s)
    display(x)
    display(doc)

'PortA'

,Unnamed: 0,Port,Currency,Date,Sys,Value,Class
0,0,PortA,AUD,2021-07-13 00:00:00+00:00,FSC,50000000,C
1,1,PortA,AUD,2021-07-13 00:00:00+00:00,FSC,400000,C
2,2,PortA,CAD,2021-07-13 00:00:00+00:00,FSC,138234286.2,C
3,3,PortA,CAD,2021-07-13 00:00:00+00:00,FSC,492384234,C
4,4,PortA,CHF,2021-07-13 00:00:00+00:00,FSC,203402,C
5,5,PortA,EUR,2021-07-13 00:00:00+00:00,FSC,2-35402950,C
6,6,PortA,EUR,2021-07-13 00:00:00+00:00,FSC,24502394,C
7,7,PortA,EUR,2021-07-15 00:00:00+00:00,FTC,22342054,C
8,8,PortA,EUR,2021-07-27 00:00:00+00:00,FTC,2349130942,C
9,9,PortA,EUR,2021-08-12 00:00:00+00:00,FTC,23489234,C


'PortB'

,Unnamed: 0,Port,Currency,Date,Sys,Value,Class
0,31,PortB,AUD,2021-07-13 00:00:00+00:00,FSC,"5,590,075.26",B
1,32,PortB,AUD,2021-08-12 00:00:00+00:00,FTC,"5,453,435.00",B
2,33,PortB,SGD,2021-07-13 00:00:00+00:00,FSC,"43,545,554.00",B
3,34,PortB,USD,2021-07-13 00:00:00+00:00,FSC,"543,543,000,000.00",B
4,35,PortB,EUR,2021-07-13 00:00:00+00:00,FSC,"54,545,435.00",B


'PortC'

,Unnamed: 0,Port,Currency,Date,Sys,Value,Class
0,36,PortC,AUD,2021-07-13 00:00:00+00:00,FSC,543545454,C
1,37,PortC,CHF,2021-07-13 00:00:00+00:00,FSC,354543545,C
2,38,PortC,CNY,2021-07-13 00:00:00+00:00,FSC,43543,C
3,39,PortC,EUR,2021-07-13 00:00:00+00:00,FSC,454545,C
4,40,PortC,HKD,2021-07-13 00:00:00+00:00,FSC,545454545,C
5,41,PortC,PLN,2021-07-13 00:00:00+00:00,FSC,11074700,C
6,42,PortC,SGD,2021-07-13 00:00:00+00:00,FSC,4546464,C
7,43,PortC,USD,2021-07-13 00:00:00+00:00,FSC,4549895+955+,C
8,44,PortC,AUD,2021-07-13 00:00:00+00:00,FSC,75174369.02,C
9,45,PortC,AUD,2021-07-15 00:00:00+00:00,FTC,261361.31,C
